In [4]:
import requests

from bs4 import BeautifulSoup

gameFILE = "C:/Users/branh/Documents/Hardwood PROJECTSSSSSS/Hardwood project/1003420UMD.html"

with open(gameFILE, "r", encoding="utf-8") as file:
    html_content = file.read()





In [5]:
soup = BeautifulSoup(html_content, 'html.parser')

#team_dic = {"shots": {"0":[0,0], "1-10":[0,0], "11-20":[0,0], "21+":[0,0]}}



inlineDivs = soup.find_all('div', style=True)

shooter_team = inlineDivs[0].text #Finds the team name of the shot chart

game_dic = {"defender_team": {"shots": {"Finishing":[0,0], "Inside Shot":[0,0], "Mid-Range":[0,0], "3-Pointer":[0,0]}}, 'players': []}

# Find all divs with inline styles
for div in inlineDivs[1:]:
    style = div['style']
    
    # Safely parse 'top' and 'left' values
    styles = {}
    for item in style.split(";"):
        if ":" in item:  # Ensure the item has a colon
            key, value = item.strip().split(":", 1)
            styles[key.strip()] = value.strip()
    
    shot_height = float(styles.get('top', 'N/A').split("px")[0]) #removes px and turns into float value
    shot_width = float(styles.get('left', 'N/A').split("px")[0]) #removes px
    
    
    # Get the span content
    span = div.find('span')

    shot_result = span.text # "x" -> miss "o" -> make
    shot_attempt = 0
    title_text = span.get("title")  # Extract the title attribute value
    title_text = title_text.replace("  ", "")

    #Get Player Names (shooter's and defender's)
    arr = title_text.split(":")
    action_string = arr[2]

    shooter_name = action_string.split("-")[0][:-1] #[:-1] to remove space in name

    if "defended by " in action_string:
        defender_string = action_string.split("defended by ")
        defender_name = defender_string[1].split(" [")[0]
    else:
        defender_name = "N/A" #no defender(for finish shots)
    
    #If the player is not in the database add them
    if not any(player["name"] == defender_name for player in game_dic["players"]):
        game_dic["players"].append({
            "name": defender_name,
            "shots": {
                "Finishing": [0, 0],
                "Inside Shot": [0, 0],
                "Mid-Range": [0, 0],
                "3-Pointer": [0, 0],
            }
        })

    
    shot_distanceSplit = title_text.split(" ")
    shot_distance = shot_distanceSplit[shot_distanceSplit.index("from") + 1] #holds shot distance 
    
    if shot_result == "o": #determines if a shot is made or not
        shot_attempt = 1

    shot_distance = int(shot_distance)
    #Paint Height and Width pixels
    paint_left, paint_right = 235, 430 # 230 <= x <= 430 shot within paint width
    paint_top = 321 # >= means shot is in paint height

    #shot_height shot_width

    #Team Defended Shots
    if shot_distance == 0 and defender_name != "N/A":
        game_dic["defender_team"]["shots"]["Finishing"][0] += shot_attempt
        game_dic["defender_team"]["shots"]["Finishing"][1] += 1

    elif shot_distance >= 21:
        game_dic["defender_team"]["shots"]["3-Pointer"][0] += shot_attempt
        game_dic["defender_team"]["shots"]["3-Pointer"][1] += 1

    elif shot_distance >= 14:
        game_dic["defender_team"]["shots"]["Mid-Range"][0] += shot_attempt
        game_dic["defender_team"]["shots"]["Mid-Range"][1] += 1

    else:
        if paint_left <= shot_width <= paint_right:
            game_dic["defender_team"]["shots"]["Inside Shot"][0] += shot_attempt
            game_dic["defender_team"]["shots"]["Inside Shot"][1] += 1
        else:
            game_dic["defender_team"]["shots"]["Mid-Range"][0] += shot_attempt
            game_dic["defender_team"]["shots"]["Mid-Range"][1] += 1

    # Update individual defender's shot statistics
    for player in game_dic["players"]:
        if player["name"] == defender_name:  # Find the defender
            if shot_distance == 0 and defender_name != "N/A":
                player["shots"]["Finishing"][0] += shot_attempt
                player["shots"]["Finishing"][1] += 1

            elif shot_distance >= 21:
                player["shots"]["3-Pointer"][0] += shot_attempt
                player["shots"]["3-Pointer"][1] += 1

            elif shot_distance >= 14:
                player["shots"]["Mid-Range"][0] += shot_attempt
                player["shots"]["Mid-Range"][1] += 1

            else:
                if paint_left <= shot_width <= paint_right:
                    player["shots"]["Inside Shot"][0] += shot_attempt
                    player["shots"]["Inside Shot"][1] += 1
                else:
                    player["shots"]["Mid-Range"][0] += shot_attempt
                    player["shots"]["Mid-Range"][1] += 1


            


        
    

    
    
    
    
    if title_text:  # Only print if the title exists
        print(f"Title: {title_text}")
   

    #print(f"Top: {top_value}, Left: {left_value}")
    
    print("-" * 50)

    
    #defended -> finish

print(game_dic)


Title: 1st Half19:37:Vernon Mays - missed shot from 5 feet defended by Brian Lay [shot clock 12 sec]
--------------------------------------------------
Title: 1st Half18:57:Neal Betz - made shot from 21 feet defended by Geraldo Munoz [shot clock 23 sec]
--------------------------------------------------
Title: 1st Half18:31:Juwan Rodgers - made shot from 21 feet defended by Ian Pruitt [shot clock 16 sec]
--------------------------------------------------
Title: 1st Half17:40:Jonathan Gorski - missed shot from 26 feet defended by Lucius Lewis [shot clock 0 sec]
--------------------------------------------------
Title: 1st Half17:10:Jonathan Gorski - made shot from 0 feet defended by Wade Murphy [shot clock 25 sec]
--------------------------------------------------
Title: 1st Half16:48:Neal Betz - made shot from 9 feet defended by Wade Murphy [shot clock 12 sec]
--------------------------------------------------
Title: 1st Half16:07:Juwan Rodgers - missed shot from 11 feet defended by Ia